<a href="https://colab.research.google.com/github/hs0224/a-professor-student-research-club/blob/main/private/Mr_Mink/Chap04_444.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 학습의 시작부터 끝까지
##### 파라미터 초기화 -> 예측 계산 -> 손실 계산 -> 그레디언트 계산 -> 가중치 한 단계 갱신 -> 과정 반복 -> 학습 종료
## 4.4.4 경사 하강법 요약
##### .....................┎> [손실 측정] -> [그레디언트 계산] ┓
##### [초기화] -> [예측] <-------------------------- [가중치 한 단계 갱신] -> [훈련 종료]
##### 시작단계(파라미터 초기화) 에서는 (1) 모델의 가중치를 임의의 값으로 설정하거나(밑바탁부터 학습), 사전에 학습된 모델로부터 설정(전이 학습)할 수 있다.
##### 손실 계산 -> '손실 함수'로 모델의 출력과 목표 타깃값을 비교한다.
##### 그레디언트 -> 미적분으로 계산, 파이토치는 자동으로 계산해준다. 또한, 경사진 정도에 '학습률'을 곱해서 '한 번에 움직여야 하는 양'을 알 수 있다.
##### 과정 반복 -> 가장 낮은 지점에 도달할 때까지 반복, 도달시 멈춘다.

## 4.5 MNIST 손실함수
#### 독립변수 x들은 이미 준비되었다. 따라서, 이를 단일 텐서로 엮어 행렬의 목록(랭크3 텐서)을 만들고, 다시 벡터의 목록(랭크2 텐서)로 바꿔준다. 이 작업에는 view 메소드를 이용한다.
##### * 파이토치의 view(): 담긴 데이터는 건드리지 않고 텐서의 모양만 바꿔줌.

In [ ]:
from fastai.vision.all import *

train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)

### torch.cat([stacked_threes, stacked_sevens]).view(-1, 28 * 28)
##### * torch.cat(): 원하는 dimension 방향으로 텐서를 나란하게 쌓는다. 기본 dimension 값은 0 이다.
##### ** view(): 텐서에 담긴 데이터는 건드리지 않고, 텐서 모양만 바꿔준다.
##### *** -1 : 해당 축을 모든 데이터에 들어맞을 만큼 크게 만든다.
##### ****28*28: 28*28 사이즈까지.

### 각 이미지에 레이블이 필요하다.
##### '3' 과 '7' 에는 각각 1과 0을 사용

In [ ]:
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_x.shape, train_y.shape

### tensor.unsqueeze()
##### * squeeze함수의 반대로 1인 차원을 생성하는 함수이다. 그래서 어느 차원에 1인 차원을 생성할 지 꼭 지정해주어야한다.
##### ** tensor.squeeze(): 차원이 1인 차원을 제거해준다. 따로 차원을 설정하지 않으면 1인 차원을 모두 제거한다. 그리고 차원을 설정해주면 그 차원만 제거한다.

### 파이토치의 Dataset은 (x,y) 튜플을 반환하기를 요구한다.

In [ ]:
deset = list(zip(train_x, train_y))
x, y = dset[0]
x.shape, y

### zip(list1, list2)
##### 리스트의 인덱스끼리 묶어 '튜플'로 구성하는 함수

In [ ]:
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x, valid_y))

### 이번에는 각 픽셀에 임의로 초기화된 가중치가 필요하다(7단계중 초기화 단계).

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

weights = init_params((28*28, 1))

### (torch.randn(size)*std).requires_grad_()
##### randn(size): 정수 랜덤값으로 초기화
##### requires_grad_(): 미분 필요 태그 삽입
### weights * pixels 형식의 함수는 충분히 유연하지 않다.
##### (픽셀값이 0이라면 곱한 결과는 항상 0이 되기 때문; 직선 방정식 'y=w*x+b'에서 b(bias)를 기억하라.)
##### b 또한 임의의 숫자로 초기화

In [ ]:
bias = init_params(1)

### y = w*x + b
##### w: 가중치(weight)
##### b: 편향(bias)
##### w 와 b 를 통틀어 파라미터(parameter)라고 한다.
### .
### 이제는 단일 이미지에 대한 예측을 계산할 수 있다.

In [ ]:
(train_x[0] * weights.T).sum() + bias